In [1]:
from acspype import ACSStream, ACSDev, ACSTSCor
from acspype.utils import find_acs_port

In [2]:
dev = ACSDev('../dev_tools/test_files/ACS-00011_2022-10-20.DEV')
tscor = ACSTSCor().to_xarray()

In [3]:
num_loops = 5
dummy_temperature = 12.345
dummy_salinity = 34.567

In [5]:
parsed_packet.sn_int

1392508939

In [4]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
with ACSStream(port) as acss: # Open a serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None:
            parsed_packet = acss.parse_packet(acs_pkt)
            calibrated_packet = acss.calibrate_packet(parsed_packet, dev)
            
            ts_corrected_packet = acss.ts_correct_packet(calibrated_packet, dummy_temperature, dummy_salinity, dev)            
            
            print(f"---------- Packet {i} ----------")
            print(f"Acquisition Time: {calibrated_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Remaining # Bytes in Buffer: {len(acss._buffer)}")
            print(f"Absorption Wavelengths: {calibrated_packet.a_wavelength}")
            print(f"Absorption With Discontinuity: {calibrated_packet.a_m_discontinuity}")
            print(f"Absorption Discontinuity Offset: {calibrated_packet.a_discontinuity_offset}")
            print(f"Measured Absorption (a_m): {calibrated_packet.a_m}")
            print(f"Absorption TS-Corrected (a_mts): {ts_corrected_packet.a_mts}")
            print('\n')
            i += 1
            if i == num_loops:
                break

---------- Packet 0 ----------
Acquisition Time: 2025-04-29T22:26:44.167Z
Remaining # Bytes in Buffer: 22
Absorption Wavelengths: [401.8 405.3 408.7 412.1 415.5 419.  423.3 427.6 431.3 435.4 439.1 442.8
 447.4 451.8 455.9 459.9 464.1 468.4 472.6 477.3 482.  486.4 490.6 494.8
 498.8 502.8 506.9 511.8 516.5 521.  524.9 529.2 532.9 537.2 541.3 545.2
 549.7 553.8 558.2 562.5 566.2 570.3 573.7 577.3 581.  584.6 586.9 590.7
 595.  599.6 603.8 608.  612.3 616.9 621.2 625.2 629.4 633.4 637.3 641.6
 646.  650.9 655.2 660.9 665.  668.5 672.9 677.1 681.1 685.5 689.2 692.6
 696.3 700.4 704.4 707.8 711.6 715.3 719.5 723.5 727.3 731.1 735.  738.9]
Absorption With Discontinuity: [-9.96286133 -9.38486525 -8.7792991  -8.21090339 -7.63549733 -7.11964678
 -6.64948203 -6.16786526 -5.71753071 -5.26538286 -4.86534323 -4.5314353
 -4.21277967 -3.91641477 -3.61850186 -3.3548021  -3.10376961 -2.87525357
 -2.65376005 -2.43713544 -2.24194649 -2.0705138  -1.9019511  -1.75756787
 -1.6310219  -1.5158191  -1.42070483